In [1]:
import re
import logging
import requests
import time
import pandas as pd

from bs4 import BeautifulSoup

logger = logging.getLogger(__name__)

In [2]:
import mysql.connector as mysql
import json

with open("./mysql_config/config.json", "r") as credential_handle:
    sql_creds = json.load(credential_handle)

db = mysql.connect(
    host     = sql_creds['host'],
    user     = sql_creds['username'],
    password = sql_creds['password'],
    use_pure = True,
    database = "sportfishing"
)

print(db)

In [ ]:
def get_table_rows(page_number):
    base_url = "https://www.fishreports.com/embed/independence-fishreports.php?page={}"
    page     = requests.get(base_url.format(page_number))
    soup     = BeautifulSoup(page.text)
    # get table rows (these contain fish report)
    rows     = soup.find_all('tr')
    
    # trim header and footer, even if the page has less than 10 posts
    rows = rows[2:]
    rows = rows[:-1]
    
    return rows

In [ ]:
def extract_fish_report(report_row):
    # get needed data from row
    date, description = report_row.find_all('td')
    date              = date.text
    main_post_link    = description.find_all('a', href=True)[0].get('href', None).replace("\\", "/")
    
    # grab the main post and extract its post text
    main_post_content = requests.get(main_post_link)
    main_post         = BeautifulSoup(main_post_content.text)
    headline          = main_post.find_all("p", attrs={'class': "text-center lead"})[0].text
    
    main_post_text    = main_post.find_all("div", attrs={"class": "report_descript_data"})[0] 
    
    # not all paragraphs are in a <p> tag for whatever reason... hack it!
    try:
        main_post_text = main_post_text.p.text.strip()
    except AttributeError:
        main_post_text = main_post_text.text.strip()
    except Exception as e:
        print(f"Failed to extract text body for {headline}, date {date}")
        raise e

    return date, headline, main_post_link, main_post_text

In [ ]:
data_rows = get_table_rows(page_number=1)

In [ ]:
extract_fish_report(data_rows[-1])

In [ ]:
# date, headline, main_post_link, main_post_text = extract_fish_report(data_rows[1])

# Add fishing reports to database

In [ ]:
# cursor = db.cursor()

# cursor.execute("SHOW DATABASES")

# ## 'fetchall()' method fetches all the rows from the last executed statement
# databases = cursor.fetchall() ## it returns a list of all databases present

# databases

# cursor.execute("SHOW TABLES")

# tables = cursor.fetchall()

# tables

In [ ]:
date, headline, main_post_link, main_post_text = extract_fish_report(data_rows[1])
date, headline, main_post_link, main_post_text

In [ ]:
def insert_fishing_report(db_obj, date, headline, post_url, post_body):
    fishing_insert_query = ("INSERT INTO fishing_reports "
                            "(date_posted, headline, post_url, post_body) " 
                            "VALUES "
                            f'(STR_TO_DATE("{date}", "%m-%d-%Y"), "{headline}", "{post_url}", "{post_body}")')
    try:
        cursor = db_obj.cursor()
        cursor.execute(fishing_insert_query)
        db_obj.commit()
        cursor.close()
    except Exception as e:
        print(f"failed to run query \n\n {fishing_insert_query} \n\n with error {e}")
        db_obj.rollback()
        print(f"failed to upload data for date {date} with headline {headline}")
    print(f"Finished for date {date} with headline {headline}")

In [ ]:
insert_fishing_report(db_obj=db, date=date, headline=headline, post_url=main_post_link, post_body=main_post_text)

In [ ]:
cursor = db.cursor()

cursor.execute("SELECT * FROM fishing_reports;")

## 'fetchall()' method fetches all the rows from the last executed statement
completed_rows = cursor.fetchall()
completed_dates = list(map(lambda x: x[1], completed_rows))
completed_dates

In [ ]:
completed_links = set(list(map(lambda x: x[3], completed_rows)))

In [ ]:
import re
def clean_string(target_string):
    return re.sub(r'\W+', ' ', target_string.replace("'", "")).strip() 

In [ ]:
t = """
"The weather was just spectacular" remarked DeBuys. \"The best times were from noon until dark on most days, but they bit some at night, too. It looked like the fish were feeding on red crab and flying fish, from what we saw in them.  There\'s an awful lot of good water headed up this way,\" continued the skipper. \"We saw a strong uphill current and just flew home on it. We found a one-degree edge about 290 miles down. The warm side was over 65 degrees and there were bluefin there. They looked like 25-pounders.\"

The group caught some 19 cows, or tuna over 200 pounds. The best one weighed 242 pounds, caught by Jim Chivas of Norwalk. He said the fish fought him for 45 minutes before it was gaffed aboard the Indy. He baited a sardine on a 6/0 Mustad Demon ringed circle hook, and used 130-pound Seaguar Premier fluorocarbon and 130-pound Spectra on a new Avet 80 reel and a Super Seeker 3 X 5 rod.

Dave Rocchi of Cypress won second place for a 239-pounder. He coaxed that one to the port bow after a half-hour tussle. His tuna bit a sardine on a 5/0 ringed Hayabusa hook on 100-pound Seaguar Premier fluorocarbon and 130-pound Line One spectra. He fished with an Avet HXW reel and a Calstar 665 XXH rod. He also had a 219-pound yellowfin.

Bill Nelson of Fairfield won third place for a 229-pound tuna. His sardine bait was pinned on a 7/0 Mustad Demon hook and 100-pound Seaguar Premier fluorocarbon, with 130-pound Kanzen spectra backing. He used a Penn 50 reel and a Seeker 6460 XXH rod to subdue the tuna after a fight of an hour and a half.

Les Nishi bagged a 224-pounder. Jim Isaac caught a brace of tuna that weighed 222 and 213 pounds. Paul Geurts had one at 219.6 pounds.

Dennis Saylor of Seal Beach bagged a triple, with his fish weighing in at 207, 203 and 201 pounds. He fished sardines on 4/0 ringed Owner Super Mutu hooks on 100-pound Momoi and 130-pound Line One spectra on an Avet Raptor reel and a Calstar 6465 XH rod.
Ev Combs of Palm Springs caught a 205-pounder with a squid under the kite. He said he used 10/0 Mustad 7691 hooks on one of the boat\'s kite rigs: 130-pound Izorline and 130-pound Izorline spectra, an Avet 80 reel and a Super Seeker 3 X 5 rod.

Chartermaster Rick Ozaki of Raider jigs and GrafTech rods got a brace, at 203 and 204 pounds. He said he baited sardines on 4/0 ringed Super Mutu hooks. He fished with 100-pound Seaguar Premier fluorocarbon and 100-pound Izorline spectra on an Avet HXW reel and a Calstar 770 XH rod.
 
Richard Berg got one at 201.8 pounds, and Dale Lethcoe snared a 201, as did Dane Barriault. Crewman Doug Brink gaffed his own 205-pound tuna, and chef Michelle, or \"Frenchie,\" as the boys call him, caught a 220-pound yellowfin.
 
The trip produced limits of tuna and wahoo, said skipper DeBuys. The best skin brought up to the scales was a 66-pound \'gator caught by Jim Mann of Bonita.  \"He bit on an orange and purple Marauder,\" said Mann, \"and he busted it.\"

This was the Indy\'s last trip of the current big fish season. She will move up to 22nd St. Landing for the next several weeks, and is expected to run short trips out to San Clemente Island and other local spots."

"""

In [ ]:
clean_string(t)

# loop over fishing report pages and scrape each (~3000 posts)

In [ ]:
# scrape_delay = 0.5

# first_page   = 25
# highest_page = 302
# final_page   = 182
# for page in range(first_page, final_page + 1):
#     if page % 5 == 0:
#         print(f"Starting page", page)
#     data_rows = get_table_rows(page_number=page)
#     for entry in data_rows:
#         time.sleep(scrape_delay)
#         try:
#             date, headline, main_post_link, main_post_text = extract_fish_report(entry)
#             if main_post_link not in completed_links:
#                 main_post_text = clean_string(main_post_text)
#                 headline       = clean_string(headline)
#                 insert_fishing_report(db_obj=db, date=date, headline=headline, post_url=main_post_link, post_body=main_post_text)
#             else:
#                 print(f"Already completed for date {date}")
#         except Exception as e:
#             print("***** ERROR ***** \n\n", entry)
#             raise e

# what fish are being talked about?

In [ ]:
example = """What a trip the guys had. Yesterday we worked our way up from down south and found some more bitting Yellowtail. Finished the day with a 100 Yellows and 5 Halibut. Great action all day. To finish up the trip this morning we rock fished and had success. Giant Reds and Lingcod for everyone. Heading home. We will be in at 0530 tomorrow. Brian & the Indy crew"""

In [ ]:
example

In [ ]:
re.findall(r"(\d+ \w+)", example)

# Begin collecting weather data

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as check
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

import time

In [15]:
weather_css_selectors = {
    "high":          "tbody.ng-star-inserted:nth-child(2) > tr:nth-child(1) > td:nth-child(2)",
    "low":           "tbody.ng-star-inserted:nth-child(2) > tr:nth-child(2) > td:nth-child(2)",
    "avg":           "tbody.ng-star-inserted:nth-child(2) > tr:nth-child(3) > td:nth-child(2)",
    "precipitation": "tbody.ng-star-inserted:nth-child(4) > tr:nth-child(1) > td:nth-child(2)",
    "visibility":    "tbody.ng-star-inserted:nth-child(8) > tr:nth-child(2) > td:nth-child(2)",
    "wind_max":      "tbody.ng-star-inserted:nth-child(8) > tr:nth-child(1) > td:nth-child(2)",
    "sea_pressure":  "tbody.ng-star-inserted:nth-child(10) > tr:nth-child(1) > td:nth-child(2)"
}

targets = ['low', 'avg', 'high', 'precipitation', 'visibility', 'wind_max', 'sea_pressure']

In [ ]:
def insert_weather_reports(db_obj, date, high, low, avg, precipitation, visibility, wind_max, sea_pressure):
    weather_insert_query = ("INSERT INTO weather_reports "
                            "(date, high_temp, low_temp, avg_temp, inches_precip, miles_visible, max_wind, sea_pressure) " 
                            "VALUES "
                            f'(STR_TO_DATE("{date}", "%Y-%m-%d"), "{high}", "{low}", "{avg}", "{precipitation}", "{visibility}", "{wind_max}", "{sea_pressure}")')
    cursor = db_obj.cursor()
    try:
        cursor.execute(weather_insert_query)
        db_obj.commit()
        print(f"Finished for date {date}")
    except Exception as e:
        print(f"failed to run query \n {weather_insert_query} \n with error {e}\n\n")
        db_obj.rollback()
        print(f"failed to upload data for date {date}")
    finally:
        cursor.close()

In [ ]:
def extract_text_css_path(driver, field):
    css_selector = weather_css_selectors[field]
    return driver.find_element_by_css_selector(css_selector).text

In [ ]:
def scrape_weather_page(weather_url):
    options = Options()
    options.headless = True

    timeout = 15
    with webdriver.Firefox(executable_path='/Users/mikelawrence/Downloads/geckodriver', options=options) as driver:
        try:
            driver.get(weather_url)
            element_present = check.presence_of_element_located((By.CSS_SELECTOR, weather_css_selectors['high']))
            WebDriverWait(driver, timeout).until(element_present)

            weather_data = {field: float(extract_text_css_path(driver, field)) for field in targets}  
        except Exception as e:
            print(f"failed to collect data for date {formatted_date}")
            raise e
            
    return weather_data

In [ ]:
weather_station_code = "KSAN"
year  = "2020"
month = "12"
day   = "15"

formatted_date = f"{year}-{month}-{day}"
formatted_date

weather_history_url = f"https://www.wunderground.com/history/daily/us/ca/san-diego/{weather_station_code}/date/{formatted_date}"

weather_data = scrape_weather_page(weather_history_url)

In [ ]:
for f, val in weather_data.items():
    print(f"{f}\t{val}")

In [ ]:
cursor = db.cursor()

cursor.execute("SELECT * FROM fishing_reports;")

## 'fetchall()' method fetches all the rows from the last executed statement
completed_rows = cursor.fetchall()
fishing_dates = set(list(map(lambda x: x[1], completed_rows)))

In [ ]:
cursor = db.cursor()

cursor.execute("SELECT * FROM weather_reports;")

## 'fetchall()' method fetches all the rows from the last executed statement
completed_rows = cursor.fetchall()
weather_dates = set(list(map(lambda x: x[1], completed_rows)))

In [ ]:
remaining_dates = fishing_dates.difference(weather_dates)
remaining_dates = list(remaining_dates)

In [ ]:
remaining_dates

In [ ]:
import time

bad_dates = []
weather_station_code = "KSAN"

date_count = len(remaining_dates)

for i, date in enumerate(remaining_dates):
    if i % 5  == 0:
        print(f"Finished {i} of {date_count}")
    day   = date.day
    month = date.month
    year  = date.year
    formatted_date = f"{year}-{month}-{day}"
    weather_history_url = f"https://www.wunderground.com/history/daily/us/ca/san-diego/{weather_station_code}/date/{formatted_date}"

    try: 
        weather_data = scrape_weather_page(weather_history_url)
        insert_weather_reports(db, formatted_date, **weather_data)
        time.sleep(5)
    except Exception as e:
        print(f"failed to get weather data for date {formatted_date} with error {e}")
        bad_dates += [formatted_date]

# Join weather and fishing data

In [14]:
'low', 'avg', 'high', 'precipitation', 'visibility', 'wind_max', 'sea_pressure'

('low',
 'avg',
 'high',
 'precipitation',
 'visibility',
 'wind_max',
 'sea_pressure')

In [16]:
targets

['low',
 'avg',
 'high',
 'precipitation',
 'visibility',
 'wind_max',
 'sea_pressure']

In [ ]:
pd.read_sql()

In [41]:
def sql_all_reports_with_weather(db_obj):
    weather_columns = ["low_temp", "avg_temp", "high_temp", 
                       "inches_precip", "miles_visible", "max_wind", 
                       "sea_pressure"]

    fishing_columns = ["date_posted", "headline", "post_body"]

    weather_columns_sql = ", ".join([f"weather_reports.{field}" for field in weather_columns])
    fishing_columns_sql = ", ".join([f"fishing_reports.{field}" for field in fishing_columns])

    join_query = f"""
    SELECT {fishing_columns_sql}, {weather_columns_sql}
    FROM fishing_reports 
    INNER JOIN weather_reports ON weather_reports.date=fishing_reports.date_posted;
    """
    return pd.read_sql(sql=join_query, con=db)
    

'weather_reports.low_temp, weather_reports.avg_temp, weather_reports.high_temp, weather_reports.inches_precip, weather_reports.miles_visible, weather_reports.max_wind, weather_reports.sea_pressure'